<a href="https://colab.research.google.com/github/MartinVIllesca/Curso-teoria-informacion/blob/master/Tarea_Capsula_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea Cápsula 0
#### EL7014 Teoría de la Información: Fundamentos y Aplicaciones <br> Information and Decision Systems Group

Nombre :

## 1. Sección Teórica

1.1. 

a) Exiplique de forma muy breve los siguientes optimizadores: 
* __SGD__: en un optimizador que actualiza los pesos solamente con la propagación del gradiente. Toma el valor del gradiente para actualizar los pesos.
* __Momentum SGD__: Agrega un término que se asocia a la historia de los gradientes, es menos sensible a grandes cambios del gradiente en un paso dependiendo de la importancia que se le dé a la historia con el valor de la constante.
* __Adagrad__: Adagrad considera un gradiente adaptivo, más bien, una actualización que pueda ser adaptiva a medida que va transcurriendo el entrenamiento. LA característica adaptiva del optimizador se relaciona con la frecuencia con que los pesos son modificados: un peso que es frecuentemente modificado no se modifica de forma considerable cada vez que se actualiza, pero un peso que es pocas veces modificado, sus modificaciones son mayores. La idea detrás de esto, es que permite que neuronas que son pocas veces actualizadas, lo sean en mayor medida cada vez, y así poder alcanzar un 
* __RMSProp__: es un método de optimización que trata de resolver el problema del decaimiento de la tasa de aprendizaje de Adagrad, esto lo hace mediante la división de la tasa de aprendizaje de Adagrad por un promedio que decae de forma exponencial de los cuadrados de los gradientes.
* __Adam__: Es un método que combina lo mejor de los optimizadores Adagrad y RMSProp.

**Nota:** Puede encontrar información útil en la siguiente fuente https://www.deeplearningbook.org/contents/optimization.html pag 290.

b) Explique a grandes rasgos las capas convolucionales y como influyen sus parámetros *stride* y *padding* tanto para el cálculo operacional como para la salida de la capa.

Las capas convolucionates son las capas que "filtran" las imágenes que entran a la misma capa. La operación consiste en pasar sobre la imagen un filtro de tamaño fijo, donde sus pesos son optimizables a través del entrenamiento de la red. El _stride_ es la cantidad de pixeles que la red se salta al mover el filtro sobre la imagen. El _padding_ es la cantidad de pixeles que se añaden a la imagen de entrada para poder operar el filtro en los bordes de la imagen, puede ser _zero padding_ o duplicar el valor del pixel del borde.

Finalmente, estas operaciones tienen el efecto de modificar la salida de la capa convolucional, donde el tamaño de la imagen puede ser inferior al de la imagen de entrada (y generalmente lo es). Donde, por ejemplo, si no hay _padding_ y hay un _stride_ de 1, además se ocupa un filtro de $3\times3$; se obtendrá una imagen de salida igual a $n-2 \times n-2$, ya que el filtro se salta los pixeles de los bordes.

**Nota:** Revisar la documentacion del modulo torch.nn.Conv2d en https://pytorch.org/docs/stable/nn.html

1.2.

Las redes neuronales pueden ser modeladas como una distribucion de probabilidad parametrizada por los pesos de la red como $T_w(x) \sim p_w(y|x)$ .
Asuma que la red neuronal tiene una funcion de salida tal que siempre su salida esta normalizada, i.e $\sum_{i=1}^C p_w(y_i|x) = 1$.<br><br>

a) Considere que dispone de $(x_i,y_i)_{i=1}^N$ datos *i.i.d.* como datos de entrenamiento. Plantee el problema de aprendizaje de una red neuronal en clasificación con el estimador de máxima verosimilitud y explicite el problema de optimización asociado. Explique intuitivamente porqué esta función objetivo permitiría tener un buen desempeño en clasificación.<br><br>

b) Sea $p(y|x)$ la distribucion real de las clases con respecto a los posible sujetos a clasificar. Utilice la Divergencia K-L para plantear el problema de aprendizaje de la red neuronal como aproximar la distribucion $p(y|x)$ con la red neuronal $T_w(x) \sim p_w(y|x)$ y explicite el problema de optimización asociado. ¿Que argumento teórico sustenta la utilización de la Divergencia K-L para plantear el problema?.
**Hint:** La divergencia K-L no es simétrica y por tanto existe una version que permite descartar terminos en el problema de optimización.<br><br>

c) Muestre que el problema de optimización asociado en el planteamiento a) es equivalente al problema de optimizacion asociado al planteamiento b). **Hint:** utilice una funcion monotona creciente bien definida entre (0,1] en la funcion objetivo del planteamiento a) para evidenciar la equivalencia entre problemas de optimizacion.

## 2. Sección Práctica



Plantee una arquitectura de red neuronal para resolver el problema de clasificacion de MNIST que cumpla con los siguientes requisitos:

* Al menos 2 capas convolucionales.
* Capas de Pooling.
* Uso del método flatten para colapsar las matrices de caracteristicas a un vector.
* Últimas capas equivalentes a una MLP (desde el flatten en adelante).

Luego entrene la arquitectura con almenos 3 optimizadores distintos y cada uno de ellos con almenos 3 set de parámetros distintos (learning rate u otro parámetro que el optimizador posea).

Reporte las curvas de Loss v/s Epoch y Accuracy v/s Epoch, luego comente muy brevemente el comportamiento de las curvas.

**Hint:** Se recomienda escribir la clase de su red convolucional como la seccion Harcoding way de la capsula 0 dado que utilizará la misma durante todas las pruebas.

### Imports

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
from matplotlib import cm

from skimage import io

import numpy as np 

print(torch.__version__)

1.4.0


In [4]:
# Asegurarse que Colab les asignó un entorno con GPU si la desean usar para entrenamiento
!nvidia-smi

Sun Apr 12 18:53:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### Descargando MNIST

MNIST es una base de datos de imagenes en blanco y negro de 28x28 pixeles, por tanto se consideran como imagenes de solo 1 canal (tonalidad de grises). Existen otras bases de datos que contienen imagenes RGB que utilizan 3 canales.

**Dato curioso:** Existen problemas de analisis de secciones saguitales del cerebro en blanco y negro que para efectos de la red consideran cada imagen de la secuencia como un canal del tensor de entrada.

In [5]:
train_dataset = datasets.MNIST(root='./', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

val_dataset = datasets.MNIST(root='./', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


### Creando Loaders

In [0]:
train_batch_size = 100
val_batch_size   = 1000

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = train_batch_size)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                          batch_size = val_batch_size)

Aqui se realiza una visualizacion de las dimensiones del tensor de entrada a nuestra red. Verificar que el orden de las dimensiones coincida con las dimensiones esperadas por los modulos convolucionales.

In [8]:
target_example, input_example = train_dataset[0]
print(target_example.shape)

torch.Size([1, 28, 28])


### Definiendo la arquitectura de la red

In [0]:
def my_first_convnet():
  def __init__(self):
        super(MLP, self).__init__()

        ## Inserte Modulo con Parámetros aqui ##
  
  def forward(x):

    ## Defina la función forward aqui ##
    
    return ... 

In [0]:
model    = my_first_convnet()

optim     = ...

criterion = ...

### Entrenamiento

In [0]:
epochs = ...

...

for e in epochs:
  
  ....